In [1]:
import cv2 as cv
import numpy as np
import random

In [37]:
img = cv.imread("task13.jpg")
temp = cv.imread("task13temp.jpg")

In [38]:
noise = 100
prev_noise = 0
result_of_ch = np.copy(img)
result_of_matching = np.copy(img)
rotated_img = np.copy(img)
bright_size = 10
contrast_size = 10
angle_size = 0.0
scale_size = 1.0
point_list=list()
point_list.append((0, 0))
point_list.append((img.shape[1], 0))
point_list.append((img.shape[1], img.shape[0]))
point_list.append((0, img.shape[0]))
point_for_change = 0

In [48]:
def update():
    global noise
    global prev_noise
    global result_of_ch
    global rotated_img
    global result_of_matching
    height, width = img.shape[:2]
    center = (width / 2, height / 2)
    if( prev_noise != noise):
        prev_noise = noise
        img_tmp = np.float64(img)  
        noise_pic = np.copy(img_tmp)
        for i in range(img_tmp.shape[0]):
            for j in range(img_tmp.shape[1]):
                noise_val = random.randint(-noise, noise)
                noise_pic[i][j] = img_tmp[i][j] + noise_val
        result_of_ch = np.uint8(np.clip(noise_pic, 0, 255))
    rotated_img = result_of_ch.copy()
    change_brightness(bright_size)
    change_contrast(contrast_size)
    rotate_matrix = cv.getRotationMatrix2D(center, angle_size, scale_size)
    rotated_img = cv.warpAffine(rotated_img, rotate_matrix, (width, height))
    matrix = cv.getPerspectiveTransform( np.float32([[0, 0], [img.shape[1], 0], [img.shape[1], img.shape[0]], [0, img.shape[0]]]), \
         np.float32(point_list))
    rotated_img = cv.warpPerspective(rotated_img, matrix, (img.shape[1], img.shape[0]), cv.INTER_CUBIC, borderMode = cv.BORDER_CONSTANT, borderValue = (0, 0, 0))
    
    imageGray = cv.cvtColor(rotated_img, cv.COLOR_BGR2GRAY)
    templateGray = cv.cvtColor(temp, cv.COLOR_BGR2GRAY)
    result = cv.matchTemplate(imageGray, templateGray,
	    cv.TM_CCOEFF_NORMED)
    (minVal, maxVal, minLoc, maxLoc) = cv.minMaxLoc(result)
    (startX, startY) = maxLoc
    endX = startX + temp.shape[1]
    endY = startY + temp.shape[0]
    result_of_matching = cv.rectangle(cv.resize(rotated_img, (int(result_of_matching.shape[1] * 0.7), int(result_of_matching.shape[0] * 0.7))), (startX, startY), (endX, endY), (255, 0, 0), 3)
    #cv.imshow("image", cv.resize(rotated_img, (int(rotated_img.shape[1] / 2), int(rotated_img.shape[0] / 2))))
    cv.imshow("result", cv.resize(result_of_matching, (int(result_of_matching.shape[1] / 2), int(result_of_matching.shape[0] / 2))))

def change_noise(val):
    global noise
    noise = val
    update()

def change_brightness( brightness):
    global rotated_img
    if brightness != 0:
        rotated_img = cv.addWeighted(rotated_img, (255 - brightness) / 255, rotated_img, 0, brightness)

def change_contrast(contrast):
    global rotated_img
    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        rotated_img = cv.addWeighted(rotated_img, f, rotated_img, 0, 127 * (1 - f))


def change_brightness_size(val):
    global bright_size
    bright_size = val
    update()


def change_contrast_size(val):
    global contrast_size
    contrast_size = val
    update()


def change_rotate_angle(val):
    global angle_size
    angle_size = val
    update()


def change_rotate_scale(val):
    global scale_size
    if val > 0:
        scale_size = val / 10.0
        update()


def change_point_for_changing(val):
    global point_for_change
    point_for_change = val


def change_point_x_val(val):
    global point_list
    _, old_y = point_list[point_for_change]
    point_list[point_for_change] = (val, old_y)
    update()   


def change_point_y_val(val):
    global point_list
    old_x, _ = point_list[point_for_change]
    point_list[point_for_change] = (old_x, val)
    update() 

    
def return_to_default(val):
    global point_list
    global scale_size
    global angle_size
    global contrast_size
    global bright_size
    scale_size = 1
    angle_size = 0
    point_list[0] = (0, 0)
    point_list[1] = (img.shape[1], 0)
    point_list[2] = (img.shape[1], img.shape[0])
    point_list[3] = (0, img.shape[0])
    update()

In [49]:
#cv.imshow("image", cv.resize(result_of_ch, (int(result_of_ch.shape[1] / 2), int(result_of_ch.shape[0] / 2))))
cv.imshow("result", cv.resize(result_of_matching, (int(result_of_matching.shape[1] / 2), int(result_of_matching.shape[0] / 2))))
cv.namedWindow('Control')
cv.createTrackbar('noise', 'Control', 100, 255, change_noise)
cv.createTrackbar('contrast', 'Control', 0, 255, change_contrast_size)
cv.createTrackbar('brightness', 'Control', 0, 255, change_brightness_size)
cv.createTrackbar('Angle', 'Control', 0, 360, change_rotate_angle)
cv.createTrackbar('Scale', 'Control', 10, 360, change_rotate_scale)
cv.namedWindow('Control2')
cv.createTrackbar('Point', 'Control2', 0, 3, change_point_for_changing)
cv.createTrackbar('X', 'Control2', 0, img.shape[1] - 1, change_point_x_val)
cv.createTrackbar('Y', 'Control2', 0, img.shape[0] - 1, change_point_y_val)
cv.createTrackbar('Return', 'Control2', 0, 1, return_to_default)
cv.waitKey(0)
cv.destroyAllWindows()